In [9]:
import requests
import json
import urllib.parse
import os
import numpy as np
import torch
import torchvision
import yaml
from PIL import Image

In [10]:
def loadData():
  # download dataset from the internet
  path = "./"
  response = requests.get('https://guillaumejaume.github.io/FUNSD/dataset.zip')
  with open(path + "dataset.zip", 'wb') as f:
    f.write(response.content)

  dataset_path = "./"
  if not os.path.exists(dataset_path):
    os.mkdir(dataset_path)
  os.system('unzip ./dataset.zip -d' + dataset_path)

  training_path = "./dataset/training_data"
  testing_path = "./dataset/testing_data"
  if not os.path.exists(os.path.join(training_path, "labels")):
    os.mkdir(os.path.join(training_path, "labels"))
  if not os.path.exists(os.path.join(testing_path, "labels")):
    os.mkdir(os.path.join(testing_path, "labels"))

  # load json file
  for json_file in list(sorted(os.listdir(os.path.join(training_path, "annotations")))):
    ann_path = os.path.join(training_path, "annotations", json_file)
    with open(ann_path) as f:
      anns = json.load(f)
      label_path = os.path.join(training_path, "labels", json_file[:-5] + ".txt")
      img_path = os.path.join(training_path, "images", json_file[:-5] + ".png")
      img_w, img_h = Image.open(img_path).size
      with open(label_path,'w') as wb:
        for ann in anns['form']:
          left = ann['box'][0]
          top = ann['box'][1]
          right = ann['box'][2]
          down = ann['box'][3]
          x = (left + right) / 2 / img_w
          y = (top + down) / 2 / img_h
          w = (right - left) / img_w
          h = (down - top) / img_h
          wb.write(str(0) + '\t' + str(x) + '\t' + str(y) + '\t' + str(w) + '\t' + str(h))
          wb.write('\n')

  for json_file in list(sorted(os.listdir(os.path.join(testing_path, "annotations")))):
    ann_path = os.path.join(testing_path, "annotations", json_file)
    with open(ann_path) as f:
      anns = json.load(f)
      label_path = os.path.join(testing_path, "labels", json_file[:-5] + ".txt")
      img_path = os.path.join(testing_path, "images", json_file[:-5] + ".png")
      img_w, img_h = Image.open(img_path).size
      with open(label_path,'w') as wb:
        for ann in anns['form']:
          left = ann['box'][0]
          top = ann['box'][1]
          right = ann['box'][2]
          down = ann['box'][3]
          x = (left + right) / 2 / img_w
          y = (top + down) / 2 / img_h
          w = (right - left) / img_w
          h = (down - top) / img_h
          wb.write(str(0) + '\t' + str(x) + '\t' + str(y) + '\t' + str(w) + '\t' + str(h))
          wb.write('\n')

In [11]:
def modelDownload():
  !git clone https://github.com/ultralytics/yolov5
  !pip install -r ./yolov5/requirements.txt
  !mv dataset yolov5

In [12]:
def run():
  loadData()
  modelDownload()

  # create yaml file
  yaml_str = """\
  path: ../yolov5/dataset  # dataset root dir
  train: training_data  # train images (relative to 'path') 128 images
  val: testing_data  # val images (relative to 'path') 128 images
  nc: 1  # number of classes
  names: ['text']  # class names
  """
  data = yaml.safe_load(yaml_str)
  with open('./yolov5/data/data.yaml', 'w') as out_data:
    yaml.dump(data, out_data, default_flow_style=False)

  # run the model
  ! python ./yolov5/train.py --img 640 --batch 10 --epochs 10 --data data.yaml --weights yolov5l.pt

In [13]:
def main():
  run()

In [14]:
if __name__ == '__main__':
  main()

Cloning into 'yolov5'...
remote: Enumerating objects: 12960, done.
remote: Total 12960 (delta 0), reused 0 (delta 0), pack-reused 12960
Receiving objects: 100% (12960/12960), 11.88 MiB | 7.98 MiB/s, done.
Resolving deltas: 100% (9002/9002), done.
     |████████████████████████████████| 596 kB 4.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


train: weights=yolov5l.pt, cfg=, data=data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=10, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-157-g7043872 torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.